# SpockFlow Decision Table Component

The Decision Table component in SpockFlow allows users to define and execute decision tables, which are structured mappings of input conditions to output values. Decision tables are particularly useful for explicit and deterministic rule-based decision-making.

### Usage

To begin using the Decision Table component, import the necessary packages and instantiate a `DecisionTable` object:


In [1]:
import pandas as pd
from spockflow.components import dtable

input_v1 = "input_v1"
input_v2 = "input_v2"

example_dt = dtable.DecisionTable()

#### Adding Conditions and Outputs

Conditions and corresponding outputs can be added to the `DecisionTable` using the `add` and `output` methods respectively. Each condition specifies an operation or comparison involving input variables, while outputs define the values or descriptions associated with matched conditions.


In [2]:
example_dt\
    .add(dtable.DTMin,   input_v1, [0,1,2,3,4,5,6,7,8,9,10])\
    .add(dtable.DTMax,   input_v1, [1,2,3,4,5,6,7,8,9,10,11])\
    .add(dtable.DTMin,   input_v2, [0,0,0,0,0,0,1,1,1,1,1])\
    .add(dtable.DTMax,   input_v2, [1,1,1,1,1,1,2,2,2,2,2])\
    .set_default(pd.DataFrame({"value": [999], "description": ["NA"]}))\
    .output("value", [1,2,0,None,-1,20,1,2,3,4,5])\
    .output("description", ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k"])


DecisionTable(operations=[DTMin(predicate=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], op='MIN'), DTMax(predicate=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], op='MAX'), DTMin(predicate=[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], op='MIN'), DTMax(predicate=[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], op='MAX')], operation_inputs=['input_v1', 'input_v1', 'input_v2', 'input_v2'], outputs={'value': [1, 2, 0, None, -1, 20, 1, 2, 3, 4, 5], 'description': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k']}, allow_multi_result=False, default_value=   value description
0    999          NA)

#### Execution

Execute the Decision Table on input data using the `execute` method:

In [3]:
input_data = pd.DataFrame({input_v1: [5, 3, 8, 0, 10], input_v2: [0,0,0,0,4]})
result_df = example_dt.execute(inputs=input_data)
result_df

,value,description
0,20.0,f
1,NaN,d
2,999.0,NA
3,1.0,a
4,999.0,NA


The `result_df` DataFrame will contain columns for each output defined, with rows corresponding to the matched conditions based on the input data.

### Saving and Loading Configurations

Similar to other components in SpockFlow, configurations of Decision Tables can be saved and loaded using configuration managers such as `YamlConfigManager`. This allows for easy deployment and reuse of decision rules.


In [4]:
from spockflow.inference.config.loader.yamlmanager import YamlConfigManager

conf_manager = YamlConfigManager()
conf_manager.save_to_config(
    model_name="demo_spock_model",
    model_version="1.0.0",
    namespace="decision_table_config",
    config=example_dt.model_dump(mode='json')
)

# Load configuration
config = conf_manager.get_config("demo_spock_model", "1.0.0")
dt_loaded = dtable.DecisionTable.from_config("decision_table_config").load(config)

# Execute with loaded configuration
result_df_loaded = dt_loaded.execute(inputs=input_data)
result_df_loaded

/Users/cp371651/.pyenv/versions/3.12.2/envs/spock/lib/python3.12/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `generator` but got `list` - serialized value may not be as expected
  Expected `generator` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


,description,value
0,f,20.0
1,d,NaN
2,NA,999.0
3,a,1.0
4,NA,999.0
